# Main parameters

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rxavier/poniard/blob/master/examples/02._main_parameters.ipynb)

This notebook outlines the most relevant options for Poniard estimators.

If you don't have it installed, please install from PyPI.

In [1]:
# %pip install poniard

At the core of Poniard lie the choice of estimators, metrics and CV strategy. While defaults might work for most cases, we try to keep it flexible.

## `estimators`

Estimators can be passed as a dict of `estimator_name: estimator_instance` or as a list of `estimator_instance`. In the latter, names will be obtained directly from the class.

Using a dictionary allows passing multiple instances of the same estimator with different hyperparameters.

In [2]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from poniard import PoniardClassifier

X, y = make_classification(n_classes=3, n_informative=3)
pnd = PoniardClassifier(estimators={"lr": LogisticRegression(max_iter=1000),
                                    "lr_no_penalty": LogisticRegression(max_iter=1000, penalty="none"),
                                    "lda": LinearDiscriminantAnalysis()})
pnd.setup(X, y)
pnd.fit()

Main metric: roc_auc_ovr
Minimum unique values to consider a number feature numeric: 10
Minimum unique values to consider a non-number feature high cardinality: 20

Inferred feature types:
    numeric categorical_high categorical_low
0       0.0                                 
1       1.0                                 
2       2.0                                 
3       3.0                                 
4       4.0                                 
5       5.0                                 
6       6.0                                 
7       7.0                                 
8       8.0                                 
9       9.0                                 
10     10.0                                 
11     11.0                                 
12     12.0                                 
13     13.0                                 
14     14.0                                 
15     15.0                                 
16     16.0                                 
1

Completed: 100%|██████████| 4/4 [00:00<00:00, 34.93it/s]


PoniardClassifier(estimators={'lr': LogisticRegression(max_iter=1000, random_state=0), 'lr_no_penalty': LogisticRegression(max_iter=1000, penalty='none', random_state=0), 'lda': LinearDiscriminantAnalysis()}, metrics=None,
    preprocess=True, scaler=standard, numeric_imputer=simple,
    custom_preprocessor=None, numeric_threshold=0.1,
    cardinality_threshold=20, cv=None, verbose=0,
    random_state=0, n_jobs=None, plugins=None,
    plot_options=PoniardPlotFactory())
            

Since we are in scikit-learn-land, most of the stuff you expect to work still works. For example, multilabel classification.

Here we had to use a dictionary because `estimator.__class__.__name__`, which is used for assigning a name to each estimator when a list is passed, would be the same for both `MultiOutputClassifier` and they would be overwritten.

In [3]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

X, y = make_multilabel_classification()
pnd = PoniardClassifier(estimators={"rf": MultiOutputClassifier(RandomForestClassifier()),
                                    "nb": MultiOutputClassifier(GaussianNB())})
pnd.setup(X, y)
pnd.fit()

Main metric: accuracy
Minimum unique values to consider a number feature numeric: 10
Minimum unique values to consider a non-number feature high cardinality: 20

Inferred feature types:
   numeric categorical_high  categorical_low
0                                        0.0
1                                        1.0
2                                        2.0
3                                        3.0
4                                        4.0
5                                        5.0
6                                        6.0
7                                        7.0
8                                        8.0
9                                        9.0
10                                      10.0
11                                      11.0
12                                      12.0
13                                      13.0
14                                      14.0
15                                      15.0
16                                      16.0
17  

Completed: 100%|██████████| 3/3 [00:01<00:00,  2.23it/s]      


PoniardClassifier(estimators={'rf': MultiOutputClassifier(estimator=RandomForestClassifier()), 'nb': MultiOutputClassifier(estimator=GaussianNB())}, metrics=None,
    preprocess=True, scaler=standard, numeric_imputer=simple,
    custom_preprocessor=None, numeric_threshold=0.1,
    cardinality_threshold=20, cv=None, verbose=0,
    random_state=0, n_jobs=None, plugins=None,
    plot_options=PoniardPlotFactory())
            

In [4]:
pnd.show_results()

,test_accuracy,train_accuracy,test_precision_macro,train_precision_macro,test_recall_macro,train_recall_macro,test_f1_macro,train_f1_macro,fit_time,score_time
nb,0.25,0.490,0.666313,0.843758,0.655411,0.818504,0.649890,0.830134,0.001661,0.002496
rf,0.22,1.000,0.687228,1.000000,0.515387,1.000000,0.542795,1.000000,0.221977,0.016469
DummyClassifier,0.09,0.065,0.208000,0.214000,0.360000,0.360000,0.262519,0.267953,0.000556,0.002127


As you may have noticed, a dummy estimator is always included even if not explicitly set during initialization.

## `metrics`

Metrics can be passed as a list of strings, following the familiar scikit-learn nomenclature, or as a dict of `str: callable`. For convenience, it can also be a single string.

This restriction is in place to facilitate naming columns in the `show_results()` method.

In [5]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from poniard import PoniardRegressor

X, y = make_regression()
pnd = PoniardRegressor(metrics=["neg_median_absolute_error", "explained_variance"],
                       estimators=[LinearRegression()])
pnd.setup(X, y)
pnd.fit()

Main metric: neg_median_absolute_error
Minimum unique values to consider a number feature numeric: 10
Minimum unique values to consider a non-number feature high cardinality: 20

Inferred feature types:
    numeric categorical_high categorical_low
0       0.0                                 
1       1.0                                 
2       2.0                                 
3       3.0                                 
4       4.0                                 
..      ...              ...             ...
95     95.0                                 
96     96.0                                 
97     97.0                                 
98     98.0                                 
99     99.0                                 

[100 rows x 3 columns]


Completed: 100%|██████████| 2/2 [00:00<00:00, 41.54it/s]


PoniardRegressor(estimators=[LinearRegression()], metrics=['neg_median_absolute_error', 'explained_variance'],
    preprocess=True, scaler=standard, numeric_imputer=simple,
    custom_preprocessor=None, numeric_threshold=0.1,
    cardinality_threshold=20, cv=None, verbose=0,
    random_state=0, n_jobs=None, plugins=None,
    plot_options=PoniardPlotFactory())
            

In [6]:
pnd.show_results()

,test_neg_median_absolute_error,train_neg_median_absolute_error,test_explained_variance,train_explained_variance,fit_time,score_time
LinearRegression,-92.524919,-2.849276e-13,7.305951e-01,1.0,0.003841,0.000677
DummyRegressor,-148.520482,-1.449634e+02,-4.440892e-17,0.0,0.001587,0.000593


In [7]:
from sklearn.metrics import r2_score, make_scorer

def scaled_r2(y_true, y_pred):
    return round(r2_score(y_true, y_pred) * 100, 1)

pnd = PoniardRegressor(metrics={"scaled_r2": make_scorer(scaled_r2, greater_is_better=True),
                                "usual_r2": make_scorer(r2_score, greater_is_better=True)},
                       estimators=[LinearRegression()])
pnd.setup(X, y).fit().show_results()

Main metric: scaled_r2
Minimum unique values to consider a number feature numeric: 10
Minimum unique values to consider a non-number feature high cardinality: 20

Inferred feature types:
    numeric categorical_high categorical_low
0       0.0                                 
1       1.0                                 
2       2.0                                 
3       3.0                                 
4       4.0                                 
..      ...              ...             ...
95     95.0                                 
96     96.0                                 
97     97.0                                 
98     98.0                                 
99     99.0                                 

[100 rows x 3 columns]


Completed: 100%|██████████| 2/2 [00:00<00:00, 92.87it/s]


,test_scaled_r2,train_scaled_r2,test_usual_r2,train_usual_r2,fit_time,score_time
LinearRegression,71.14,100.0,0.711461,1.0,0.001807,0.000289
DummyRegressor,-3.44,0.0,-0.034588,0.0,0.000642,0.000261


## `cv`

Cross validation can be anything that scikit-learn accepts. By default, classification tasks will be paired with a `StratifiedKFold` if the target is binary, and `KFold` otherwise. Regression tasks use `KFold` by default.

`cv=int` or `cv=None` are internally converted to one of the above classes so that Poniard's `random_state` parameter can be passed on.

In [8]:
from IPython.utils import io
from sklearn.model_selection import RepeatedKFold

with io.capture_output() as c:
    pnd_5 = PoniardRegressor(cv=4).setup(X, y)
    pnd_none = PoniardRegressor(cv=None).setup(X, y)
    pnd_k = PoniardRegressor(cv=RepeatedKFold(n_splits=3)).setup(X, y)

In [9]:
print(pnd_5.cv_, pnd_none.cv_, pnd_k.cv_, sep="\n")

KFold(n_splits=4, random_state=0, shuffle=True)
KFold(n_splits=5, random_state=0, shuffle=True)
RepeatedKFold(n_repeats=10, n_splits=3, random_state=0)


Note that even though we didn't specify `random_state` for the third estimator, it gets injected during setup.